In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import layers,models
import tensorflow as tf
import re,string
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding



In [ ]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
df.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [ ]:
x = df['comment_text']
y = df.drop(columns=['id','comment_text']).values

In [ ]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
def preprocess_text(text):
    text = tf.strings.lower(text)  # Convert to lowercase
    text = tf.strings.regex_replace(text, '[%s]' % re.escape(string.punctuation), '')  # Remove punctuation
    return text

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x, y)) #Create a TensorFlow Dataset
dataset = dataset.map(lambda x, y: (preprocess_text(x), y))


In [ ]:
#splitting data in tensorflow pipeline
train_size = int(0.8 * len(df))
val_size = int(0.1 * len(df))  # Reserve 10% for validation
train = dataset.take(train_size).batch(32).shuffle(buffer_size=1000).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val = dataset.skip(train_size).take(val_size).batch(32).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test = dataset.skip(train_size + val_size).batch(32).cache().prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=10000, output_sequence_length=1800)
vectorize_layer.adapt(dataset.map(lambda x, y: x))#This method learns the vocabulary from the training data. It uses only the comments (x) for adapting the vectorization, ensuring the layer knows how to map text to integers.

In [ ]:
train_dataset = train.map(lambda x, y: (vectorize_layer(x), y))
test_dataset = test.map(lambda x, y: (vectorize_layer(x), y))
val = val.map(lambda x, y: (vectorize_layer(x), y))


model

In [ ]:
model = models.Sequential([
    layers.Embedding(input_dim=10001, output_dim=32, input_length=100),  # Set input_length based on your input data
    layers.Bidirectional(LSTM(32,activation='tanh')),
    layers.Dense(128,activation='relu'),
    layers.Dense(256,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(6,activation='sigmoid')
])
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [52]:
history = model.fit(test_dataset, epochs=2, validation_data=val)
history

Epoch 1/2
499/499 ━━━━━━━━━━━━━━━━━━━━ 991s 2s/step - loss: 0.0448 - val_loss: 0.0639
Epoch 2/2
499/499 ━━━━━━━━━━━━━━━━━━━━ 971s 2s/step - loss: 0.0391 - val_loss: 0.0710


In [ ]:
input_text1 = preprocess_text('your black stay away')
input_text1=vectorize_layer(input_text1)
input_text1 = tf.keras.preprocessing.sequence.pad_sequences([input_text1], maxlen=100, padding='post')
input_text1 = tf.convert_to_tensor(input_text1)


In [ ]:
p=model.predict(input_text)
p

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step


array([[0.06311927, 0.00015037, 0.01115219, 0.00132622, 0.01157023,
        0.00318218]], dtype=float32)

In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [54]:
batch_X = preprocess_text(batch_X)
batch_X = vectorize_layer(batch_X)
batch_X = tf.keras.preprocessing.sequence.pad_sequences(batch_X, maxlen=100, padding='post')

In [60]:
l=(model.predict(batch_X) > 0.5).astype(int)
l

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [59]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [62]:
model.evaluate(batch_X,batch_y)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 638ms/step - loss: 0.0578


0.057845860719680786